In [5]:
import numpy as np

## Task 1
Implement the encryptor for a simplified AES-like cipher with the parameters given in the previous slides and the following substitution function:
    $$f : y_j(j): 2v_i(j) \mod  p, \ j$$

In [6]:
def subkey_generator(key):

    k_1 = [key[0],key[2],key[4],key[6]] 
    k_2 = [key[0],key[1],key[2],key[6]] 
    k_3 = [key[0],key[3],key[4],key[6]] 
    k_4 = [key[0],key[3],key[5],key[6]] 
    k_5 = [key[0],key[2],key[5],key[6]] 
    k_6 = [key[2],key[4],key[4],key[6]] 

    return [k_1, k_2, k_3, k_4, k_5, k_6]

def subkey_sum(k_i,w,p):

    k_inter = k_i+k_i
    k = np.array(k_inter)

    return (w + k)%p



In [7]:
 
def timesTwo(v,p):

    return np.multiply(v,2)%p

def transposition(y):

    return np.array([y[0],y[1],y[2],y[3],y[7],y[6],y[5],y[4]])

def linear(z, p):

    z_matrix = z.reshape(2,4)

    param_matrix = np.matrix([[2,5],[1,7]])

    w_matrix = np.dot(param_matrix,z_matrix)%p

    w_array = np.asarray(w_matrix).flatten()

    return w_array
        




In [8]:
def encryption(u,k,p,round,substitution,transposition,subkey_sum,linear):

    k_list = subkey_generator(k)

    for i in range (round):
        
        #print("round : ",i)
        v = subkey_sum(k_list[i],u,p)         
        #print("post subkey_sum",v)
        y = substitution(v,p)
        #print("post timesTwo",y)
        z = transposition(y)
        #print("post transpo",z)
        if i == round-1:
            x = subkey_sum(k_list[i+1],z,p)
            break
        u = linear(z, p)
        #print("post linear", u)
        
    return x


In [9]:
u = np.array([1,0,0,0,0,0,0,0])
k = np.array([1,0,0,0,0,0,0,0])
p=11
r = 5

x = encryption(u,k,p,r,
               substitution=timesTwo,
               transposition=transposition,
               subkey_sum=subkey_sum,
               linear=linear)

print(x)


[4 0 0 9 7 0 0 3]


## Task 2
Implement the decryptor for this simplified AES-like cipher. Note that decryption is performed by the inverse blocks in reverse order. Therefore, you have to implement the inverse of each function used to encrypt the message (subkey sum, substitution, transposition and linear), taking into consideration that all the operations must be done in the field $\mathbb{F} = GF(p)$.

In [ ]:
def inv_timesTwo(v,p):
    mul_inv = pow(2,-1,p)
    return np.multiply(v,mul_inv)%p


def inv_transposition(y):
    return np.array([y[0],y[1],y[2],y[3],y[7],y[6],y[5],y[4]])


def inv_subkey_sum(k_i,w,p):

    k_inter = k_i+k_i
    k = np.array(k_inter) #list

    return (w - k)%p

#Calcolo dell'inversa della matrice modulo p
def inv_matrix(m, p):
    det = int(np.round(np.linalg.det(m)))  #Determinante della matrice
    det_inv = pow(det, -1, p)  #Inverso del determinante modulo p
    adj_matrix = np.array([[m[1, 1], -m[0, 1]],
                           [-m[1, 0], m[0, 0]]])  #Matrice aggiunta
    m_inv = (det_inv * adj_matrix) % p  #Inversa modulo p

    return m_inv

def inv_linear(z, p):
    z_matrix = z.reshape(2, 4)

    param_matrix = np.array([[2, 5], [1, 7]])

    #Calcolo dell'inversa della matrice modulo p
    param_matrix_inv = inv_matrix(param_matrix, p)

    w_matrix = np.dot(param_matrix_inv, z_matrix) % p

    w_array = np.asarray(w_matrix, dtype=int).flatten()

    return w_array


def decryption(x,k,p,round):
    k_list = subkey_generator(k)

    for i in range(round,-1,-1):
        print("round : ", i)

        z = inv_subkey_sum(k_list[i],x,p)            #v = u - k
        print("post subkey_sum",z)
        
        if i == 0:
            x = z
            break
        elif i != round:
            w = inv_linear(z, p)
            print("post linear", w)
        else:
            w = z

        y = inv_transposition(w)
        print("post transpo",y)

        x = inv_timesTwo(y,p)
        print("post timesTwo",x)
        
    return x

Test for inv_linear

In [11]:
a = linear(np.array([1,0,3,5,0,0,8,0]), p)  #change the array to test
print(a)
b = inv_linear(a, p)
print(b)

[ 2  0  2 10  1  0  4  5]
[1 0 3 5 0 0 8 0]


In [12]:
u = decryption(x,k,p,r)
print(u)

round :  5
post subkey_sum [4 0 0 9 7 0 0 3]
post transpo [4 0 0 9 3 0 0 7]
post timesTwo [ 2  0  0 10  7  0  0  9]
round :  4
post subkey_sum [ 1  0  0 10  6  0  0  9]
post linear [6 0 0 4 0 0 0 7]
post transpo [6 0 0 4 7 0 0 0]
post timesTwo [3 0 0 2 9 0 0 0]
round :  3
post subkey_sum [2 0 0 2 8 0 0 0]
post linear [2 0 0 4 4 0 0 1]
post transpo [2 0 0 4 1 0 0 4]
post timesTwo [1 0 0 2 6 0 0 2]
round :  2
post subkey_sum [0 0 0 2 5 0 0 2]
post linear [ 7  0  0  9  6  0  0 10]
post transpo [ 7  0  0  9 10  0  0  6]
post timesTwo [ 9  0  0 10  5  0  0  3]
round :  1
post subkey_sum [ 8  0  0 10  4  0  0  3]
post linear [4 0 0 0 0 0 0 2]
post transpo [4 0 0 0 2 0 0 0]
post timesTwo [2 0 0 0 1 0 0 0]
round :  0
post subkey_sum [1 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0]


Other tests for decryption

In [13]:
a = np.array([1,8,3,6,5,6,7,2]) #change plaintext to test
k = np.array([1,0,4,0,3,0,2,0]) #change key to test
x = encryption(u,k,p,r,
               substitution=timesTwo,
               transposition=transposition,
               subkey_sum=subkey_sum,
               linear=linear)

print(b)
c = decryption(b,k,p,r)
print(c)

[1 0 3 5 0 0 8 0]
round :  5
post subkey_sum [8 8 0 3 7 8 5 9]
post transpo [8 8 0 3 9 5 8 7]
post timesTwo [ 4  4  0  7 10  8  4  9]
round :  4
post subkey_sum [3 0 0 5 9 4 4 7]
post linear [ 1 10 10  0  9  7  7  1]
post transpo [ 1 10 10  0  1  7  7  9]
post timesTwo [ 6  5  5  0  6  9  9 10]
round :  3
post subkey_sum [5 5 5 9 5 9 9 8]
post linear [ 6  5  5  5  3 10 10  2]
post transpo [ 6  5  5  5  2 10 10  3]
post timesTwo [3 8 8 8 1 5 5 7]
round :  2
post subkey_sum [2 8 5 6 0 5 2 5]
post linear [ 4  1  4  8  1 10  6  9]
post transpo [ 4  1  4  8  9  6 10  1]
post timesTwo [ 2  6  2  4 10  3  5  6]
round :  1
post subkey_sum [1 6 9 2 9 3 1 4]
post linear [8 3 4 3 8 0 9 8]
post transpo [8 3 4 3 8 9 0 8]
post timesTwo [ 4  7  2  7  4 10  0  4]
round :  0
post subkey_sum [ 3  3 10  5  3  6  8  2]
[ 3  3 10  5  3  6  8  2]


## Task 3
Identify the overall linear relationship for this simplified AES-like cipher, that is find the
matrices $A ∈ F^{(ℓ_x × ℓ_k)}$ and $B ∈ F^{(ℓ_x × ℓ_u)}$ such that
$$x = E(k, u) = Ak + Bu \mod p$$
with all operations in the field $\mathbb{F} = GF(p)$.

In [ ]:
e = np.eye(8, dtype=int)
k_a = np.zeros(8)
A = np.empty((8,0))

x = encryption(u,k,p,r,
               substitution=timesTwo,
               transposition=transposition,
               subkey_sum=subkey_sum,
               linear=linear)


for i in range(8):

    encrypted_value = encryption(k_a, np.asarray(e[i]).flatten(), p, r,
                                 substitution=timesTwo,
                                 transposition=transposition,
                                 subkey_sum=subkey_sum,
                                 linear=linear)
    
    A = np.hstack((A, encrypted_value.reshape(-1, 1)))

print(A)



k_b = np.eye(8, dtype=int)
e = np.zeros(8)
B = np.empty((8,0))

for i in range(8):

    encrypted_value = encryption(np.asarray(k_b[i]).flatten(),e,p,r,
                                 substitution=timesTwo,
                                 transposition=transposition,
                                 subkey_sum=subkey_sum,
                                 linear=linear)
    B = np.hstack((B, encrypted_value.reshape(-1, 1)))


print(B)

x = (np.dot(A,k) + np.dot(B,u))%p

print(x)

[[ 9.  0.  1.  0.  0.  0.  6.  0.]
 [ 0.  8.  6.  1.  3.  9.  0.  0.]
 [ 0.  6.  0.  8.  3. 10.  0.  0.]
 [ 6.  0.  0.  0.  0.  0. 10.  0.]
 [ 2.  0.  1.  0.  0.  0.  3.  0.]
 [ 0.  1.  8.  3. 10.  6.  0.  0.]
 [ 0. 10.  0.  5.  7.  6.  0.  0.]
 [ 3.  0.  0.  0.  0.  0.  3.  0.]]
[[6. 0. 0. 3. 3. 0. 0. 0.]
 [0. 6. 3. 0. 0. 3. 0. 0.]
 [0. 3. 6. 0. 0. 0. 3. 0.]
 [3. 0. 0. 6. 0. 0. 0. 3.]
 [5. 0. 0. 0. 4. 0. 0. 8.]
 [0. 5. 0. 0. 0. 4. 8. 0.]
 [0. 0. 5. 0. 0. 8. 4. 0.]
 [0. 0. 0. 5. 8. 0. 0. 4.]]
[ 9.  0.  9.  7.  6.  7. 10.  9.]


## Task 5
implement the encryptor for a simplified AES-like cipher with the parameters given in the previous slides and the substitution function described by the following table
|$v_i(j)$|0|1|2|3|4|5|6|7|8|9|10|
|--------|-|-|-|-|-|-|-|-|-|-|--|
|$y_i(j)$|0|2|4|8|6|10|1|3|5|7|9|

where $j ∈ \{1, . . . , ℓ\}$

In [15]:
def custom_substitution_number(v):
    if(v == 0):
        return 0
    if(v == 1):
        return 2
    if(v == 2):
        return 4
    if(v == 3):
        return 8
    if(v == 4):
        return 6
    if(v == 5):
        return 10
    if(v == 6):
        return 1
    if(v == 7):
        return 3
    if(v == 8):
        return 5
    if(v == 9):
        return 7
    if(v == 10):
        return 9
    
def custom_substitution(v,p):

    for i in range(8):
        v[i] = custom_substitution_number(v[i])
    
    return v


def encryption_v2(u,k,p,round):

    k_list = subkey_generator(k)

    for i in range (round):
        
        #print("round : ",i)
        v = subkey_sum(k_list[i],u,p)
        #print("post subkey_sum",v)
        y = custom_substitution(v,p)
        #print("post custom substitution",y)
        z = transposition(y)
        #print("post transpo",z)
        if i == round-1:
            x = subkey_sum(k_list[i+1],z,p)
            break
        u = linear(z, p)
        #print("post linear", u)
        
    return x


In [16]:
print(encryption(u,k,p,r,
                 substitution=custom_substitution,
                 transposition=transposition,
                 subkey_sum=subkey_sum,
                 linear=linear
                 ))

[9 8 0 7 6 2 7 9]


## Task 7
implement the encryptor for a simplified AES-like cipher with the following parameters: $\mathcal{K} = \mathbb{F}^{\ell_k}, \ell_k = 4$

ECC...

In [17]:
def subkey_generator_v2(key):

    k_1 = [key[0],key[1],key[2],key[3]]
    k_2 = [key[0],key[1],key[3],key[2]]
    k_3 = [key[1],key[2],key[3],key[0]]
    k_4 = [key[0],key[3],key[1],key[2]]
    k_5 = [key[2],key[3],key[0],key[1]]
    k_6 = [key[1],key[3],key[0],key[2]]

    return [k_1,k_2,k_3,k_4,k_5,k_6]

def mod_inv(v, p):
    
    x =[]

    for num in v:
        if(num != 0):
            x.append(2*pow(num, -1, p))
        else:
            x.append(0)


    return np.array(x) # Calcola l'inverso di x modulo p

def substitution(x):
    x= x.tolist()
    return mod_inv(x, p)
    

In [18]:
def encryption_v3(u,k,p,round):

    k_list = subkey_generator_v2(k)

    for i in range (round):
        
        #print("round : ",i)
        v = subkey_sum(k_list[i],u,p)
        #print("post subkey_sum",v)
        y = substitution(v)
        #print("post custom substitution",y)
        z = transposition(y)
        #print("post transpo",z)
        if i == round-1:
            x = subkey_sum(k_list[i+1],z,p)
            break
        u = linear(z, p)
        #print("post linear", u)
        
    return x

In [19]:
k=np.array([1,0,0,0])

p= 11

print(encryption_v3(u,k,p,r))

[5 0 3 2 5 2 1 1]
